In [1]:
import pandas as pd

In [2]:
user = pd.read_csv('datasets/sirius/user_1.csv')

In [3]:
user_session = pd.read_csv('datasets/sirius/user_session.csv')

## Задача 1

Посчитайте долю пользователей мужского пола в таблице `user`.

Ответ округлите до $2$-х знаков после запятой.

**Рекомендации по решению**

Для подсчёта доли объектов, удовлетворяющих заданному условию, удобно использовать функцию `mean`, вызвав её от колонки-фильтра со значениями булевого типа. В данном случае каждое знаечние `True` будет приведено к $1$, а каждое значение `False` — к $0$. Усреднение этих значений как раз и даст долю объектов, удволетворяющих заданному условию.

In [4]:
(user.gender.value_counts()['М'] / len(user))

0.425

## Задача 2

Посчитайте долю пользователей из Краснодарского края, которые заходили на платформу в сентябре $2023$-го года.

Ответ округлите до $2$-х знаков после запятой.

In [5]:
kras = user[(user['region'] == 'Краснодарский край')]['id'].count()
kras_2309 = user[(user['region'] == 'Краснодарский край') & (user['last_visited'].str.startswith('2023-09'))]['id'].count()
kras_2309 / kras

0.41007194244604317

## Задача 3

По таблице `user_session` рассчитайте долю активных пользователей в мае $2023$-го года относительно общего числа пользователей платформы из таблицы `user`.

Активными считайте пользователей, у которых в этот месяц была хотя бы одна сессия.

Ответ округлите до $2$-х знаков после запятой.

In [6]:
users_count = user['id'].count()
active_sessions_0523 = user_session[user_session.date.str.startswith('2023-05')]
active_users_0523 = active_sessions_0523.groupby('user_id').agg({'user_id': pd.Series.nunique}).count()['user_id']
active_users_0523 / users_count

0.598

## Задача 4

Дадим другое определение активным пользователям. Будем считать активными в конкретном месяце тех, кто посмотрел хотя бы $60$ минут видеоконтента в этом месяце.

Рассчитайте долю активных пользователей среди тех, у кого была хотя бы одна сессия в марте $2023$-го года.

Ответ округлите до $2$-х знаков после запятой.

In [7]:
user_session.head()

,session_id,user_id,date,time_watched
0,1,558,2023-01-23 00:52:14,10
1,2,435,2023-01-06 20:59:45,51
2,3,783,2023-01-27 19:24:25,2
3,4,575,2023-01-23 21:55:54,29
4,5,598,2023-01-22 15:06:09,5


In [8]:
sessions_0323 = user_session[user_session.date.str.startswith('2023-03')]
users_0323 = sessions_0323.groupby('user_id').agg({'user_id': pd.Series.nunique}).count()['user_id']
userstime_0323 = sessions_0323.groupby('user_id').agg({'time_watched': 'sum'})
active_users = userstime_0323[userstime_0323.time_watched >= 60].count()['time_watched']
active_users / users_0323

0.9192872117400419

# Задачи 5-9

Рассчитайте, сколько в среднем смотрят видеоконтента пользователи за один день активности. Днём активности называем день, в который у пользователя была сессия.

## Задача 5

Для этого сначала вычислите суммарное число дней, когда пользователи смотрели видеоконтент.

Обратите внимание, что хараткеристику можно посчитать двумя способами. Рассчитатйте её как число уникальных пар вида (пользователь; день, когда у него была сессия).

In [9]:
sessions = user_session.copy()
sessions.date = pd.to_datetime(sessions.date).dt.date

In [10]:
active_days_per_user = sessions.groupby('user_id').agg({'date': pd.Series.nunique})
total_active_days = active_days_per_user.date.sum()
total_active_days

44932

## Задача 6

Теперь вычислите суммарное время просмотра видеоконтента пользователями.

In [11]:
total_time_watched = sessions.time_watched.sum()
total_time_watched

1607552

## Задача 7

Вычислите среднее количество просмотренного пользователями видеоконтента за день активности, как отношение суммарного времени просмотра к суммарному числу дней просмотра.

Ответ округлите до $2$-х знаков после запятой.

In [12]:
total_time_watched / total_active_days

35.77744146710585

## Задача 8

Обратите внимание, что есть альтернативный способ рассчитать, сколько в среднем видеоконтента смотрят пользователи за один день активности.

Во-первых, рассчитайте для каждого из пользователей среднее количество просмотренного им видеоконтента. Принцип вычисления метрики для конкретного пользователя такой же, как был для всех пользователей вместе: нужно взять суммарное время просмотра контента и разделить на суммарное количество уникальных дней активности.

В качестве ответа укажите среднее количество просмотренного видеоконтента пользователем с `id`, равным $738$. Ответ округлите до $2$-х знаков после запятой.

**Рекомендации по решению**

Для того чтобы вычислить среднее количество видеоконтента, просмотренного конкретным пользователем, удобно провести группировку по пользователям и произвести агрегацию по двум колонкам отдельно: по колонке `time_watched`, просуммировав значения в ней , и по колонке `date`, посчитав количество уникальных дней активности для каждого из пользователей.

Для того чтобы произвести агрегацию по колонке `date`, удобно воспользоваться лямбда-функцией, которая сначала произведёт округление моментов времени из колонке `date` _вниз_ до дней, а потом посчитает количество полученных уникальных дней.

In [17]:
sessions.head(3)

,session_id,user_id,date,time_watched
0,1,558,2023-01-23,10
1,2,435,2023-01-06,51
2,3,783,2023-01-27,2


In [30]:
time_per_user = sessions.groupby('user_id').agg({'time_watched': 'sum'})
days_per_user = sessions.groupby('user_id').agg({'date': pd.Series.nunique})
time_per_user.loc[738]['time_watched'] / days_per_user.loc[738]['date']

40.42424242424242

## Задача 9

Для того чтобы вычислить среднее время просмотра видеоконтента за день активности, посчитайте среднее арифметическое данной характеристики по всем пользователям. Для этого используйте данные, полученные в предыдущем упражнении.

Ответ округлите до $2$-х знаков после запятой.

In [49]:
time_days_watched = time_per_user.join(days_per_user).rename(columns={'time_watched': 'total_time', 'date': 'total_days'})
time_days_watched['mean_time'] = time_days_watched.total_time / time_days_watched.total_days
time_days_watched.mean_time.sum() / len(time_days_watched)

34.793012875603004